In [1]:
!pip install gym==0.25.2
!pip install swig
!pip install gym[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 52.7 MB/s eta 0:00:00
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2376095 sha256=286bf9e4149e5199247a4fe673a69b734912e6b841db7c9cacbf472cbb28e504
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py
  Attempting uninstall: pygame
    Found existing installation: pygame 2.5.2
    Uninstalling pygame-2.5.2:
      Successfully uninstalled pygame-2.5.2


In [2]:
PATH = '/content/drive/MyDrive/Pytorch/rl/BDQN'

In [3]:
import os
os.chdir(PATH)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
from collections import deque
import gym

# Define the Bayesian Q-Network
class BayesianQNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=128, dropout_prob=0.2):
        super(BayesianQNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.dropout = nn.Dropout(p=dropout_prob)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        return self.fc3(x)

# Define the BDQN Agent with Double Q-Learning
class BDQNAgent:
    def __init__(self, state_size, action_size, hidden_size=128, dropout_prob=0.2, learning_rate=0.0005,
                 gamma=0.99, batch_size=128, buffer_size=100000, tau=0.01, update_freq=4, n_samples=10):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        self.batch_size = batch_size
        self.buffer = deque(maxlen=buffer_size)
        self.tau = tau
        self.update_freq = update_freq
        self.n_samples = n_samples

        self.qnetwork_local = BayesianQNetwork(state_size, action_size, hidden_size, dropout_prob).to(device)
        self.qnetwork_target = BayesianQNetwork(state_size, action_size, hidden_size, dropout_prob).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=learning_rate)

        self.t_step = 0

    def step(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
        self.t_step = (self.t_step + 1) % self.update_freq
        if len(self.buffer) > self.batch_size and self.t_step == 0:
            experiences = self.sample()
            self.learn(experiences)

    def act(self, state, eps=0.):
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = [self.qnetwork_local(state) for _ in range(self.n_samples)]
            action_values = torch.stack(action_values).mean(0)
        self.qnetwork_local.train()

        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def sample(self):
        experiences = random.sample(self.buffer, k=self.batch_size)
        states, actions, rewards, next_states, dones = zip(*experiences)

        states = torch.FloatTensor(states).to(device)
        actions = torch.LongTensor(actions).to(device)
        rewards = torch.FloatTensor(rewards).to(device)
        next_states = torch.FloatTensor(next_states).to(device)
        dones = torch.FloatTensor(dones).to(device)

        return (states, actions, rewards, next_states, dones)

    def learn(self, experiences):
        states, actions, rewards, next_states, dones = experiences

        Q_targets_next = [self.qnetwork_local(next_states).detach() for _ in range(self.n_samples)]
        Q_targets_next = torch.stack(Q_targets_next).mean(0)
        Q_targets_next = Q_targets_next.gather(1, self.qnetwork_target(next_states).max(1)[1].unsqueeze(1)).squeeze(1)

        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))

        Q_expected = self.qnetwork_local(states).gather(1, actions.unsqueeze(1)).squeeze(1)

        loss = F.mse_loss(Q_expected, Q_targets)

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.qnetwork_local.parameters(), 1)  # Gradient clipping
        self.optimizer.step()

    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

    def save(self, filename):
        torch.save(self.qnetwork_local.state_dict(), filename)

    def load(self, filename, map_location=None):
        self.qnetwork_local.load_state_dict(torch.load(filename, map_location=map_location))
        self.qnetwork_target.load_state_dict(torch.load(filename, map_location=map_location))


    def update_target_network(self):
        self.qnetwork_target.load_state_dict(self.qnetwork_local.state_dict())

# Define the training loop
def train(env, agent, n_episodes=3000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    scores = []
    scores_window = deque(maxlen=100)
    eps = eps_start

    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break
        scores_window.append(score)
        scores.append(score)
        eps = max(eps_end, eps_decay * eps)
        print(f"Episode: {i_episode} Reward: {score}")
        # print(type(i_episode))
        if i_episode % 50 == 0:

            agent.update_target_network()
            agent.save('BDQN.pth')
            print(f'\nTarget network updated at episode {i_episode}')

        if i_episode > 10 and sum(scores[-5:])>1000:
            agent.save('BDQN.pth')
            print(f'\nEnvironment solved in {i_episode} episodes!')
            break



    return scores

# Initialize environment and agent
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
env = gym.make('LunarLander-v2')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = BDQNAgent(state_size, action_size)

path = 'BDQN.pth'
try:
    map_location = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    agent.load(path, map_location=map_location)
    print("Checkpoint loaded from file.")

except FileNotFoundError:
    print("No checkpoint found, starting from scratch.")


# Train the agent
scores = train(env, agent)


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


No checkpoint found, starting from scratch.


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 1 Reward: -202.15461808886963


<ipython-input-4-03a3bf79932a>:69: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  states = torch.FloatTensor(states).to(device)


Episode: 2 Reward: -114.63086700450455
Episode: 3 Reward: -409.9351329529532
Episode: 4 Reward: -91.79632719961323
Episode: 5 Reward: -465.1984060588341
Episode: 6 Reward: -433.37134523378467
Episode: 7 Reward: -273.8003444330608
Episode: 8 Reward: -73.13331767968961
Episode: 9 Reward: -240.66010189045264
Episode: 10 Reward: -110.37967886091687
Episode: 11 Reward: -197.34201147537766
Episode: 12 Reward: -105.21659706625539
Episode: 13 Reward: -203.47485939051674
Episode: 14 Reward: -127.78776322164124
Episode: 15 Reward: -146.59319263072052
Episode: 16 Reward: -129.23203984167765
Episode: 17 Reward: -333.1524572670096
Episode: 18 Reward: -466.0948795142107
Episode: 19 Reward: -248.0258495993663
Episode: 20 Reward: -177.808462795193
Episode: 21 Reward: -77.85214373273324
Episode: 22 Reward: -292.76559857240966
Episode: 23 Reward: -229.13910771295042
Episode: 24 Reward: -239.55025228348393
Episode: 25 Reward: -44.151952078211934
Episode: 26 Reward: -174.05993209989344
Episode: 27 Reward:

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
